## This notebook
For this notebook we will be writing the nuts and bolts of a Python package. It will be easiest to follow along with the instructions if you start your jupyter session from inside the `Notebooks/` directory. 

## Objects, Packages, and Classes
Before completing this notebook first complete your reading of Chapters 14-18 in [ThinkPython](http://www.greenteapress.com/thinkpython2/html/index.html). Once completed, you will now have a fairly complete background in Python programming. You are familiar with the core tools that you need to write almost any program in Python. We'll still continue learning many more tricks in the coming weeks for working with statistical packages and making code run efficiently, but now the more important thing to learn is how to put the pieces that we've learned so far together into coherent ways to solve problems. 

### Python packages
One major goal of scientific programming is to create a tool that is not only sufficient to achieve your goal, but is generally useful enough that it can be reused by others to achieve the same goal as well. For this, you'll need to write your code so that it is user-friendly and easy to understand, and you'll also need to package it in a way that it is easy to access. These two goals are the focus of this notebook. 

### Imports
We've learned already how to import packages using the `import` statement, and we know that the packages we are importing have either been part of the standard Python library, or else we installed them ourselves using `conda`. But what does a Python package look like? Well, it's just a collection of Python code, of course. But there is a specific structure to how it is written that allows it to be imported. So let's cover that now.

In [2]:
## import installed packages
import toyplot
import random


### Local imports
The first step towards writing a Python package is to save your Python functions into a `.py` file. We created one in class by copying the functions we wrote in a notebook into a separate file using the text editor built into Jupyter. There are many possible text editors you could use. And we discussed the specific structure of `.py` files in the lecture. Take a look now at the file we wrote in class that is in `Notebooks/debruijn_funcs.py`. Similarly, you can take a look at the file `Notebooks/eulerian.py`. Our file `debruijn_funcs.py` defines several functions, while the file `eulerian.py` defines several functions in addition to a few Class objects, which we'll discuss below and were covered in your reading. Both of these are valid `.py` files. Because of this, we can use `import` on either of them to gain access to their Classes and functions here in this notebook. So let's try that below. This is called a 'local import' because we are importing these files from our current directory (`Notebooks/`). We'll cover later how to make them importable from anywhere. 

In [3]:
## this imports all functions from debuijn.py
import debruijn_funcs

## this imports just one function from eulerian.py
from eulerian import eulerian_path

## Classes
Classes are an interesting aspect of Python coding, as they are not actually essential the way that functions or data types are, but stylistically they are quite beneficial to learn and use as they make for a very organized way to write and package your code. Here we're going to rewrite our code from the debruijn scripts so that it is organized into a Class object. 

### Planning the code design 
There are many ways to structure a Class object, so keep in mind that the design we've decided to use here is not the only way that this could be done. Many aspects of designing a Class object are about aesthetics, and the way in which you want a user to interact with your code. We'll walk through several steps below to write our Class piece by piece. First, though, we'll define our objective using the commented code below as for how we want the Class to be structured:

#### Attributes of the object
Attributes are typically stored objects that the user may want to view or interact with. These are commonly strings, tuples, lists, etc. For our object, we want to have the following attributes accessible to the user. These correspond to the randomly generated target sequence, the randomly generated sequenced reads, the kmers generated from those reads, the edges of the deBruijn graph, and the assembly sequence made from that graph. 

In [4]:
# obj.target
# obj.reads
# obj.kmers
# obj.edges
# obj.assembly

#### Functions of the object
We will create seven functions for the object. The special function `__init__()` is automatically called when a Class object is created. The next five functions we have seen before, and will be copied more or less from our `debruijn_funcs.py` script, and the last function `run()` we will write here. As you can see the debruijn functions are all written with an underscore before their names. This means that we are designating them as "private" functions. They are meant for the developer to use and know about, but are not expected to be called by the user. The underscore essentially hides them from being seen when you use tab-completion. Only the `.run()` function will be exposed to the user.  

In [5]:
# obj.__init__()
# obj._random_dna()
# obj._get_reads()
# obj._get_kmers()
# obj._reads_to_kmers()
# obj._get_debruijn_edges()
# obj._get_eulerian_path()
# obj.run()

So a user would then interact with it by doing something like the code below. We know assembling the debruijn graph and inferring the eulerian path requires calling several different functions. So we'll make the `.run()` function call all of these subfunctions for us. 

In [6]:
## initialize an Assembly object for a target that is 500bp
#data = Assembly(500)

## return .assembly result, takes input of nreads, rlen, and k
## and calls many subfunctions to perform assembly
#data.run(nreads=50, rlen=20, k=4)

### Step 1 in coding the Assembly Class object
I've started writing the Class object for you. The first thing we should look for to try to understand what a Class object does is to look at the `__init__()` function. This is the function that is automatically executed when an object of this class is created. Here we can see that the `__init__()` function takes one required argument (target_length) and one optional argument (random_seed). The second argument is optional because there is a default value entered for it in the function arguments (random_seed=123). 

You can see that the `__init__` function defines an attribute `.target` and it also calls a function called `_random_sequence`, which is defined below. The `_random_sequence` function is copied from the `debruijn_funcs` scripts, but is modified slightly. Here, instead of having a `return` statement to return the value, it instead *stores its results to the target attribute of self*. To make `_random_sequence` a function of self, we need to add `self` as the first argument to its function. This is simply the syntax of how to define Class functions. We've also added the function call `random.seed` to initialize the random number generator using the `random_seed` variable. This will make our analyses reproducible. 

In [7]:
class Assembler1():
    """
    An object for constructing a debuijn graph from kmers of random reads
    """
    def __init__(self, target_length, random_seed=123):

        ## store attributes
        self.target = None #The output is stored here (initialize an empty output)
        
        ## run init functions
        random.seed(random_seed)
        self._random_sequence(target_length)
        
        
    def _random_sequence(self, target_length):
        self.target = "".join((random.choice("ACGT") for i in range(target_length)))

Let's test out our Class object so far by initializing an `instance`, which we could name anything. I'll call it data throughout this notebook. 


In [8]:
## init an instance and print the target
data = Assembler1(500)
data.target

'AGATGAATGGACCGGCCATATAAGTAAACCAGTTGTAGGTCGATTTTGACATGCGGTATTGACAGAGCTAGTCTCCTTTAACTCAGGGTTAAAGAATATATAGGTAGTGACTACGCAATGCGTGCGACTTACTAGTTGTACGCGAGCGAGCCGGTAACCACGCAGAGCCTTTAGTGTCCATGAAGTGTGGCTGCCTATTGCTGAGAGGATATGATGCTGTGCTGGACAATTCTATCGGCATGAGTCAGGCTGTAATCCAGCCGTTCGAATGCATTTAATTGCTCCCCACCATAATGAACTGCGAGGCAGCTCTCTTCTATAGCTACATGATGGGCTAAAGACTCTCTCACAGAACCTTCTAACATACAGTCCCTGCAAGGTGCAAAACGCCAGTGGCTAGGGAGAGATCGTACTACCGTTGCTAGACCGCACAGAAGAGGCGTAGAAGGGTCAAAATTGCCTATCCATCACGTCTGAAATGACGGGACGGCTCTATACCA'

### Step 2 in the Assembly object
The other functions that are outside of `__init__()` will need to be called from the object after it is initialized. This is a good way to structure our object since we want to be able to initialize a single target sequence of DNA and then perhaps try many different parameters for the number of reads and their length and kmer size to see how they affect our ability to assemble it. 


A good way to write your Class is to define all of the attributes at the top of the Class, and then to define the functions below. So let's add a few more attributes here that we wish to define eventually, and we'll add one more function to fill in one of the attributes. Again, the functions here are being taken from the debruijn funcs we wrote before, and just modified slightly. You can see that now the `_get_reads` function doesn't require us to input the `target` sequence to it anymore, but instead since its first argument is `self`, it can access the target sequence directly from self. And, once again, instead of having a return value it stores its results directly to the `self.reads` attribute. 

In [9]:
class Assembler2():
    """
    An object for constructing a debuijn graph from kmers of random reads
    """
    def __init__(self, target_length, random_seed=123):

        ## store attributes
        self.target = None 
        self.reads = None
        self.kmers = None
        self.edges = None
        self.assembly = None
        
        ## run init functions
        random.seed(random_seed)
        self._random_sequence(target_length)
        
        
    ## private functions
    def _random_sequence(self, target_length):
        self.target = "".join((random.choice("ACGT") for i in range(target_length)))
        
    
    def _get_reads(self, nreads, rlen):
        "returns nreads of len rlen drawn from string"  
        last_start = len(self.target) - rlen
        startpoints = [random.randint(0, last_start) for i in range(nreads)]
        self.reads = [self.target[i:i+rlen] for i in startpoints]
        

In [10]:
## init an instance of the Class
data = Assembler2(200)

## test the private function _get_reads
data._get_reads(10, 20)
data.reads

['TTTGACATGCGGTATTGACA',
 'GAGCGAGCCGGTAACCACGC',
 'TATATAGGTAGTGACTACGC',
 'AATATATAGGTAGTGACTAC',
 'ATATAAGTAAACCAGTTGTA',
 'CTTTAACTCAGGGTTAAAGA',
 'GTTGTAGGTCGATTTTGACA',
 'CTAGTCTCCTTTAACTCAGG',
 'TTACTAGTTGTACGCGAGCG',
 'TAACCACGCAGAGCCTTTAG']

### Step 3 in the Assembly object

Now we're going to write a composite function that calls several private functions. The reason for this is that we don't really have a need for stopping in between running these functions, so it is best to write a function that calls all three of them. Alternatively, you could imagine just writing a single function to perform all three tasks, but that will generally make the code harder to read and less clear. Here we will add `_get_kmers` and `_reads_to_kmers` and define a new function `run` that runs three functions together. 

In [11]:
class Assembler3():
    """
    An object for constructing a debuijn graph from kmers of random reads
    """
    def __init__(self, target_length, random_seed=123):

        ## store attributes
        self.target = None 
        self.reads = None
        self.kmers = None
        self.edges = None
        self.assembly = None
        
        ## run init functions
        random.seed(random_seed)
        self._random_sequence(target_length)
        
        
    ## private functions
    def _random_sequence(self, target_length):
        self.target = "".join((random.choice("ACGT") for i in range(target_length)))
        
        
    def _get_reads(self, nreads, rlen):
        "returns nreads of len rlen drawn from string"  
        last_start = len(self.target) - rlen
        startpoints = [random.randint(0, last_start) for i in range(nreads)]
        self.reads = [self.target[i:i+rlen] for i in startpoints]
        
        
    def _get_kmers(self, string, k): # This is how you get your k-mers once you get your reads 
        "returns k-mers dict for a string target"
        kmers = {}
        for i in range(0, len(string) - k + 1):
            kmer = string[i:i+k]
            if kmer in kmers:
                kmers[kmer] += 1
            else:
                kmers[kmer] = 1
        return kmers
    

    def _reads_to_kmers(self, k):
        "stores kmers to dict uses update to join together kmer dict keys"
        kmers = {}
        for read in self.reads:
            ikmers = self._get_kmers(read, k)
            kmers.update(ikmers)
        self.kmers = kmers
        
        
    ## public function
    def run(self, nreads, rlen, k):
        "generates reads and breaks them into kmers"
        self._get_reads(nreads, rlen)
        self._reads_to_kmers(k)
        

In [12]:
## init an instance of the Class
data = Assembler3(200)

## test the private function _get_reads
data.run(50, 50, 5)

## check out kmers
len(data.kmers)
data.kmers

{'AAACC': 1,
 'AAAGA': 1,
 'AACCA': 1,
 'AACTC': 1,
 'AAGAA': 1,
 'AAGTA': 1,
 'AAGTG': 1,
 'AATAT': 1,
 'AATGC': 1,
 'AATGG': 1,
 'ACAGA': 1,
 'ACATG': 1,
 'ACCAC': 1,
 'ACCAG': 1,
 'ACCGG': 1,
 'ACGCA': 1,
 'ACGCG': 1,
 'ACTAC': 1,
 'ACTAG': 1,
 'ACTCA': 1,
 'ACTTA': 1,
 'AGAAT': 1,
 'AGAGC': 1,
 'AGCCG': 1,
 'AGCCT': 1,
 'AGCGA': 1,
 'AGCTA': 1,
 'AGGGT': 1,
 'AGGTA': 1,
 'AGGTC': 1,
 'AGTAA': 1,
 'AGTCT': 1,
 'AGTGA': 1,
 'AGTGT': 2,
 'AGTTG': 1,
 'ATAAG': 1,
 'ATAGG': 1,
 'ATATA': 1,
 'ATGAA': 1,
 'ATGCG': 1,
 'ATGGA': 1,
 'ATTGA': 1,
 'ATTTT': 1,
 'CAATG': 1,
 'CACGC': 1,
 'CAGAG': 1,
 'CAGGG': 1,
 'CAGTT': 1,
 'CATAT': 1,
 'CATGA': 1,
 'CATGC': 1,
 'CCACG': 1,
 'CCAGT': 1,
 'CCATA': 1,
 'CCATG': 1,
 'CCGGC': 1,
 'CCGGT': 1,
 'CCTAT': 1,
 'CCTTT': 1,
 'CGACT': 1,
 'CGAGC': 2,
 'CGATT': 1,
 'CGCAA': 1,
 'CGCAG': 1,
 'CGCGA': 1,
 'CGGCC': 1,
 'CGGTA': 1,
 'CGTGC': 1,
 'CTACG': 1,
 'CTAGT': 1,
 'CTATT': 1,
 'CTCAG': 1,
 'CTCCT': 1,
 'CTGCC': 1,
 'CTTAC': 1,
 'CTTTA': 1,
 'GAAGT': 1,

### Step 4 in writing the Assembly object
Now as you see below we add two additional functions to the `.run()` function, and we define both of those functions just above it. Here I've changed the code slightly since we wrote it in class to fix some bugs. The `get_debruijn_edges` function finds kmers than overlap by n-1 of their length, and the `eulerian_path` function looks for a unique path through the debruijn graph that touches each edge once. The eulerian function here also concatenates the returned path into a string. The eulerian_path function raises an error if a single path is not found, so we add an exception clause to simply return an empty string if the path cannot be found. 

In [27]:
class Assembler4():
    """
    An object for constructing a debuijn graph from kmers of random reads
    """
    def __init__(self, target_length, random_seed=123): # self and random_seed are ignored, but essentially the target_length is called when you call the Assembler

        ## store attributes
        self.target = None 
        self.reads = None
        self.kmers = None
        self.edges = None
        self.assembly = None
        
        ## run init functions
        random.seed(random_seed)
        self._random_sequence(target_length)
        
        
    ## private functions
    def _random_sequence(self, target_length):
        self.target = "".join((random.choice("ACGT") for i in range(target_length)))
        
        
    def _get_reads(self, nreads, rlen):
        "returns nreads of len rlen drawn from string"  
        last_start = len(self.target) - rlen
        startpoints = [random.randint(0, last_start) for i in range(nreads)]
        self.reads = [self.target[i:i+rlen] for i in startpoints]
        
        
    def _get_kmers(self, string, k):
        "returns k-mers dict for a string target"
        kmers = {}
        for i in range(0, len(string) - k + 1):
            kmer = string[i:i+k]
            if kmer in kmers:
                kmers[kmer] += 1
            else:
                kmers[kmer] = 1
        return kmers
    

    def _reads_to_kmers(self, k):
        "stores kmers to dict uses update to join together kmer dict keys"
        kmers = {}
        for read in self.reads:
            ikmers = self._get_kmers(read, k)
            kmers.update(ikmers)
        self.kmers = kmers
        


    def _get_debruijn_edges(self):
        " return edges of the debruijn graph for a set of kmers"
        edges = set() # Sets are immutable
        kmers = tuple(self.kmers.keys())
        for k1 in kmers:
            for k2 in kmers:
                ## if xaa = aax then add (xaa, aax)
                if k1[1:] == k2[:-1]:
                    edges.add((k1, k2)) # Only adding them into a set if they overlap by n-1
        self.edges = edges

        
        
    def _get_eulerian_path(self):
        """
        returns eulerian path through kmers joined as a string.
        Uses the loaded 'eulerian_path()' function from eulerian.py
        """
        try:
            epath = eulerian_path(self.edges)
            path = epath[0]
            for kmer in epath[1:]:
                path += kmer[-1]
            self.assembly = path
        except Exception: # global exception 
            self.assembly = ""
          
        
    ## public function
    def run(self, nreads, rlen, k):
        "generates reads and breaks them into kmers"
        self._get_reads(nreads, rlen)
        self._reads_to_kmers(k)
        self._get_debruijn_edges()
        self._get_eulerian_path()
        

### Test our full assembler
Now we can test our Class object easily with just a few commands. Having a simple object like this also makes it much easier to write methods for testing it. 

In [14]:
## init an Assembler object
data = Assembler4(200) # The final target sequence we are trying to get at

## run assembly for given parameters
data.run(300, 50, 15) # Here, we assign the number of reads, the length of each read and the length of each kmer

## ask whether assembly matches the target
data.assembly == data.target # Then we test if the assembly matches the final target sequence

True

### Test our assembler over different values
Here we use a for-loop to test over many different parameters to see how they affect our ability to assemble the target sequence. 

In [15]:
result_dict = {} # Testing over different combinations to see if it matches the assembly. 

for target_size in [200, 500, 1000]: # range of different target sizes
    data = Assembler4(target_size)
    
    for nreads in [500, 1000, 5000]:
        for k in [10, 20, 30]:
            data.run(nreads=nreads, rlen=50, k=k)
            
            ## store result in dict
            result = data.assembly == data.target # Value is TRUE or FALSE
            result_dict[(target_size, nreads, k)] = result  

### It's harder to assemble longer targets with fewer reads or shorter kmers

In [16]:
result_dict

{(200, 500, 10): True,
 (200, 500, 20): True,
 (200, 500, 30): True,
 (200, 1000, 10): True,
 (200, 1000, 20): True,
 (200, 1000, 30): True,
 (200, 5000, 10): True,
 (200, 5000, 20): True,
 (200, 5000, 30): True,
 (500, 500, 10): False,
 (500, 500, 20): False,
 (500, 500, 30): True,
 (500, 1000, 10): False,
 (500, 1000, 20): True,
 (500, 1000, 30): True,
 (500, 5000, 10): True,
 (500, 5000, 20): True,
 (500, 5000, 30): True,
 (1000, 500, 10): False,
 (1000, 500, 20): False,
 (1000, 500, 30): False,
 (1000, 1000, 10): False,
 (1000, 1000, 20): False,
 (1000, 1000, 30): True,
 (1000, 5000, 10): True,
 (1000, 5000, 20): True,
 (1000, 5000, 30): True}

## Assignment

#### (1) Add a new function to the Assembler Class object called `test()`
This function should call the `.run()` function over a range of parameter values in a for-loop like we did above and return the results as a dictionary. Test that your function works and returns results similar to the example above. 

In [45]:
class Assembler5():
    """
    An object for constructing a debuijn graph from kmers of random reads
    """
    def __init__(self, target_length, random_seed=123): # self and random_seed are ignored, but essentially the target_length is called when you call the Assembler

        ## store attributes
        self.target = None 
        self.reads = None
        self.kmers = None
        self.edges = None
        self.assembly = None
        
        ## run init functions
        random.seed(random_seed)
        self._random_sequence(target_length)
        
        
    ## private functions
    def _random_sequence(self, target_length):
        self.target = "".join((random.choice("ACGT") for i in range(target_length)))
        
        
    def _get_reads(self, nreads, rlen):
        "returns nreads of len rlen drawn from string"  
        last_start = len(self.target) - rlen
        startpoints = [random.randint(0, last_start) for i in range(nreads)]
        self.reads = [self.target[i:i+rlen] for i in startpoints]
        
        
    def _get_kmers(self, string, k):
        "returns k-mers dict for a string target"
        kmers = {}
        for i in range(0, len(string) - k + 1):
            kmer = string[i:i+k]
            if kmer in kmers:
                kmers[kmer] += 1
            else:
                kmers[kmer] = 1
        return kmers
    

    def _reads_to_kmers(self, k):
        "stores kmers to dict uses update to join together kmer dict keys"
        kmers = {}
        for read in self.reads:
            ikmers = self._get_kmers(read, k)
            kmers.update(ikmers)
        self.kmers = kmers
        


    def _get_debruijn_edges(self):
        " return edges of the debruijn graph for a set of kmers"
        edges = set() # Sets are immutable
        kmers = tuple(self.kmers.keys())
        for k1 in kmers:
            for k2 in kmers:
                ## if xaa = aax then add (xaa, aax)
                if k1[1:] == k2[:-1]:
                    edges.add((k1, k2)) # Only adding them into a set if they overlap by n-1
        self.edges = edges

        
        
    def _get_eulerian_path(self):
        """
        returns eulerian path through kmers joined as a string.
        Uses the loaded 'eulerian_path()' function from eulerian.py
        """
        try:
            epath = eulerian_path(self.edges)
            path = epath[0]
            for kmer in epath[1:]:
                path += kmer[-1]
            self.assembly = path
        except Exception: # global exception 
            self.assembly = ""
          
     
     
    ## public function
    def run(self, nreads, rlen, k):
        "generates reads and breaks them into kmers"
        self._get_reads(nreads, rlen)
        self._reads_to_kmers(k)
        self._get_debruijn_edges()
        self._get_eulerian_path()
        
    def test(self, p1, p2, p3):
    
        result_dict= {}
    
        for target_size in [p1,p2,p3] :
            data = Assembler5(target_size)
        
            for nreads in [500,1000,5000] :
                for k in [10,20,30]:
                    data.run(nreads=nreads,rlen=50,k=k)
                
                    result = data.assembly == data.target
                    result_dict[(target_size,nreads,k)] = result
                    
        return result_dict
                    
      

In [47]:
data = Assembler5(300)

data.test(200,500,1000)



{(200, 500, 10): True,
 (200, 500, 20): True,
 (200, 500, 30): True,
 (200, 1000, 10): True,
 (200, 1000, 20): True,
 (200, 1000, 30): True,
 (200, 5000, 10): True,
 (200, 5000, 20): True,
 (200, 5000, 30): True,
 (500, 500, 10): False,
 (500, 500, 20): False,
 (500, 500, 30): True,
 (500, 1000, 10): False,
 (500, 1000, 20): True,
 (500, 1000, 30): True,
 (500, 5000, 10): True,
 (500, 5000, 20): True,
 (500, 5000, 30): True,
 (1000, 500, 10): False,
 (1000, 500, 20): False,
 (1000, 500, 30): False,
 (1000, 1000, 10): False,
 (1000, 1000, 20): False,
 (1000, 1000, 30): True,
 (1000, 5000, 10): True,
 (1000, 5000, 20): True,
 (1000, 5000, 30): True}

#### (2) Add a new function to the Assembler Class object called `plot()`
This function should return a toyplot graph object using the function below, or some modification of it if you feel like tweaking the parameters to make the graph look better. You can learn more about the toyplot plotting library [here](https://toyplot.rtfd.io) if you'd like. Test that your plot function works and draws a graph. I would advise testing it on relatively small graphs (e.g., target_length < 300) otherwise it starts to get messy and memory intensive. 

In [63]:
class Assembler6():
    """
    An object for constructing a debuijn graph from kmers of random reads
    """
    def __init__(self, target_length, random_seed=123): # self and random_seed are ignored, but essentially the target_length is called when you call the Assembler

        ## store attributes
        self.target = None 
        self.reads = None
        self.kmers = None
        self.edges = None
        self.assembly = None
        
        ## run init functions
        random.seed(random_seed)
        self._random_sequence(target_length)
        
        
    ## private functions
    def _random_sequence(self, target_length):
        self.target = "".join((random.choice("ACGT") for i in range(target_length)))
        
        
    def _get_reads(self, nreads, rlen):
        "returns nreads of len rlen drawn from string"  
        last_start = len(self.target) - rlen
        startpoints = [random.randint(0, last_start) for i in range(nreads)]
        self.reads = [self.target[i:i+rlen] for i in startpoints]
        
        
    def _get_kmers(self, string, k):
        "returns k-mers dict for a string target"
        kmers = {}
        for i in range(0, len(string) - k + 1):
            kmer = string[i:i+k]
            if kmer in kmers:
                kmers[kmer] += 1
            else:
                kmers[kmer] = 1
        return kmers
    

    def _reads_to_kmers(self, k):
        "stores kmers to dict uses update to join together kmer dict keys"
        kmers = {}
        for read in self.reads:
            ikmers = self._get_kmers(read, k)
            kmers.update(ikmers)
        self.kmers = kmers
        


    def _get_debruijn_edges(self):
        " return edges of the debruijn graph for a set of kmers"
        edges = set() # Sets are immutable
        kmers = tuple(self.kmers.keys())
        for k1 in kmers:
            for k2 in kmers:
                ## if xaa = aax then add (xaa, aax)
                if k1[1:] == k2[:-1]:
                    edges.add((k1, k2)) # Only adding them into a set if they overlap by n-1
        self.edges = edges

        
        
    def _get_eulerian_path(self):
        """
        returns eulerian path through kmers joined as a string.
        Uses the loaded 'eulerian_path()' function from eulerian.py
        """
        try:
            epath = eulerian_path(self.edges)
            path = epath[0]
            for kmer in epath[1:]:
                path += kmer[-1]
            self.assembly = path
        except Exception: # global exception 
            self.assembly = ""
          
    def _plot(self):
        e0 = [i[0] for i in self.edges]
        e1 = [i[1] for i in self.edges]
        toyplot.graph(e0, e1, tmarker=">", vlstyle={'font-size': '8px'});  
     
    ## public function
    def run(self, nreads, rlen, k):
        "generates reads and breaks them into kmers"
        self._get_reads(nreads, rlen)
        self._reads_to_kmers(k)
        self._get_debruijn_edges()
        self._get_eulerian_path()
        self._plot()
        
    def test(self, p1, p2, p3):
    
        result_dict= {}
    
        for target_size in [p1,p2,p3] :
            data = Assembler6(target_size)
        
            for nreads in [500,1000,5000] :
                for k in [10,20,30]:
                    data.run(nreads=nreads,rlen=50,k=k)
                
                    result = data.assembly == data.target
                    result_dict[(target_size,nreads,k)] = result
                    
        return result_dict
                    
    


In [64]:
data = Assembler6(200)
data.run(200, 50, 15)

AAACCAGTTGTAGGT AAAGAATATATAGGT AACCACGCAGAGCCT AACCAGTTGTAGGTC AACTCAGGGTTAAAG AAGAATATATAGGTA AAGTAAACCAGTTGT AAGTGTGGCTGCCTA AATATATAGGTAGTG AATGCGTGCGACTTA AATGGACCGGCCATA ACAGAGCTAGTCTCC ACATGCGGTATTGAC ACCACGCAGAGCCTT ACCAGTTGTAGGTCG ACCGGCCATATAAGT ACGCAATGCGTGCGA ACGCAGAGCCTTTAG ACGCGAGCGAGCCGG ACTACGCAATGCGTG ACTAGTTGTACGCGA ACTCAGGGTTAAAGA ACTTACTAGTTGTAC AGAATATATAGGTAG AGAGCCTTTAGTGTC AGAGCTAGTCTCCTT AGCCGGTAACCACGC AGCCTTTAGTGTCCA AGCGAGCCGGTAACC AGCTAGTCTCCTTTA AGGGTTAAAGAATAT AGGTAGTGACTACGC AGGTCGATTTTGACA AGTAAACCAGTTGTA AGTCTCCTTTAACTC AGTGACTACGCAATG AGTGTCCATGAAGTG AGTGTGGCTGCCTAT AGTTGTACGCGAGCG AGTTGTAGGTCGATT ATAAGTAAACCAGTT ATAGGTAGTGACTAC ATATAAGTAAACCAG ATATAGGTAGTGACT ATATATAGGTAGTGA ATGAAGTGTGGCTGC ATGAATGGACCGGCC ATGCGGTATTGACAG ATGCGTGCGACTTAC ATGGACCGGCCATAT ATTGACAGAGCTAGT ATTTTGACATGCGGT CAATGCGTGCGACTT CACGCAGAGCCTTTA CAGAGCCTTTAGTGT CAGAGCTAGTCTCCT CAGGGTTAAAGAATA CAGTTGTAGGTCGAT CATATAAGTAAACCA CATGAAGTGTGGCTG CATGCGGTATTGACA CCACGCAGAGCCTTT CCAGTTGTAGGTCGA CCATATAAGTAAACC CCATGAAGTGTGGCT CCGGCCATATAAGTA CCGGTAACCACGCAG CCTTTAACTCAGGGT CCTTTAGTGTCCATG CGACTTACTAGTTGT CGAGCCGGTAACCAC CGAGCGAGCCGGTAA CGATTTTGACATGCG CGCAATGCGTGCGAC CGCAGAGCCTTTAGT CGCGAGCGAGCCGGT CGGCCATATAAGTAA CGGTAACCACGCAGA CGGTATTGACAGAGC CGTGCGACTTACTAG CTACGCAATGCGTGC CTAGTCTCCTTTAAC CTAGTTGTACGCGAG CTCAGGGTTAAAGAA CTCCTTTAACTCAGG CTTACTAGTTGTACG CTTTAACTCAGGGTT CTTTAGTGTCCATGA GAAGTGTGGCTGCCT GAATATATAGGTAGT GAATGGACCGGCCAT GACAGAGCTAGTCTC GACATGCGGTATTGA GACCGGCCATATAAG GACTACGCAATGCGT GACTTACTAGTTGTA GAGCCGGTAACCACG GAGCCTTTAGTGTCC GAGCGAGCCGGTAAC GAGCTAGTCTCCTTT GATGAATGGACCGGC GATTTTGACATGCGG GCAATGCGTGCGACT GCAGAGCCTTTAGTG GCCATATAAGTAAAC GCCGGTAACCACGCA GCCTTTAGTGTCCAT GCGACTTACTAGTTG GCGAGCCGGTAACCA GCGAGCGAGCCGGTA GCGGTATTGACAGAG GCGTGCGACTTACTA GCTAGTCTCCTTTAA GGACCGGCCATATAA GGCCATATAAGTAAA GGGTTAAAGAATATA GGTAACCACGCAGAG GGTAGTGACTACGCA GGTATTGACAGAGCT GGTCGATTTTGACAT GGTTAAAGAATATAT GTAAACCAGTTGTAG GTAACCACGCAGAGC GTACGCGAGCGAGCC GTAGGTCGATTTTGA GTAGTGACTACGCAA GTATTGACAGAGCTA GTCCATGAAGTGTGG GTCGATTTTGACATG GTCTCCTTTAACTCA GTGACTACGCAATGC GTGCGACTTACTAGT GTGTCCATGAAGTGT GTGTGGCTGCCTATT GTTAAAGAATATATA GTTGTACGCGAGCGA GTTGTAGGTCGATTT TAAACCAGTTGTAGG TAAAGAATATATAGG TAACCACGCAGAGCC TAACTCAGGGTTAAA TAAGTAAACCAGTTG TACGCAATGCGTGCG TACGCGAGCGAGCCG TACTAGTTGTACGCG TAGGTAGTGACTACG TAGGTCGATTTTGAC TAGTCTCCTTTAACT TAGTGACTACGCAAT TAGTGTCCATGAAGT TAGTTGTACGCGAGC TATAAGTAAACCAGT TATAGGTAGTGACTA TATATAGGTAGTGAC TATTGACAGAGCTAG TCAGGGTTAAAGAAT TCCATGAAGTGTGGC TCCTTTAACTCAGGG TCGATTTTGACATGC TCTCCTTTAACTCAG TGAAGTGTGGCTGCC TGAATGGACCGGCCA TGACAGAGCTAGTCT TGACATGCGGTATTG TGACTACGCAATGCG TGCGACTTACTAGTT TGCGGTATTGACAGA TGCGTGCGACTTACT TGGACCGGCCATATA TGTACGCGAGCGAGC TGTAGGTCGATTTTG TGTCCATGAAGTGTG TGTGGCTGCCTATTG TTAAAGAATATATAG TTAACTCAGGGTTAA TTACTAGTTGTACGC TTAGTGTCCATGAAG TTGACAGAGCTAGTC TTGACATGCGGTATT TTGTACGCGAGCGAG TTGTAGGTCGATTTT TTTAACTCAGGGTTA TTTAGTGTCCATGAA TTTGACATGCGGTAT TTTTGACATGCGGTA

#### (3) Copy our Assembler Class object to a new .py file
Once your Assembler Class function has been fully tested copy/paste the Class object into a new text file and save it as `Notebooks/{username}_dbClass.py`. (Note: if your username has a `-` character in it then do not include this in your filename, it will cause problems). Make sure that you format this `.py` file as we've discussed, by putting a shebang at the top, followed by comments, then imports, then code. Do not forget to add import statements for both eulierian.py and toyplot. Check that you can import the file once you've saved it by calling `import` on it below (this is the reason I'm having you save it in Notebooks/ currently).  

In [1]:
## edit this to the name of *your* script
import vjjan91_dbClass

#### (4) Finally, copy your .py file to the `Assignments/` directory and make pull request
Add, commit and push to your repository and make a pull request on the Course repo to submit your assignment. 